# Setup

In [ ]:
# Image from Drive
from google.colab import drive      
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --quiet -U --no-cache-dir gdown --pre
!pip install --quiet nlp-id
!pip install --quiet emoji
!pip install --quiet imblearn
!pip install --quiet gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 65.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from nlp_id.stopword import StopWord
from nlp_id.postag import PosTag
from nlp_id.lemmatizer import Lemmatizer

import emoji
import string
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')

import seaborn as sns # untuk visualisasi
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

# potongan kode di bawah adalah agar gambar graph yang 
# dihasilkan mempunyai kualitas yang cukup baik
sns.set(rc={"figure.dpi":100, 'savefig.dpi':300})
sns.set_context('notebook')
sns.set_style("ticks")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Load Data

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/re_dataset.csv', encoding='latin-1')
kamus_alay = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/kamusalay.csv', encoding='latin-1')

test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/test_data.csv', encoding='latin-1')
synthetic_test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/synthetic_test_data_5x.csv', encoding='latin-1')
# train data
remaining_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/remaining_data.csv', encoding='latin-1')
eda_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/eda_sample.csv', encoding='latin-1')

potensi_bias = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/potensi_bias.csv', encoding='latin-1')
most_freq = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/most_freq_words.csv', encoding='latin-1')
abusive_lexicon = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/abusive.csv', encoding='latin-1')

# Feature Extraction

In [ ]:
# Tokenization & Emoticon Processor

regex_str = []
regex_str.append(r'<[^>]+>') # HTML tags
regex_str.append(r'(?:@[\w_]+)') # @-mentions
regex_str.append(r'(?:&[\w_]+)')
regex_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
regex_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
regex_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers
regex_str.append(r"(?:[a-z][a-z'\-_]+[a-z])") # words with - and '
regex_str.append(r'(?:[\w_]+)') # other words
regex_str.append(r'(?:\S)') # anything else

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

def tokenize(sentence):
    return [emoji.demojize(term) for term in tokens_re.findall(sentence)]

In [ ]:
# Stopwords Removal & Lemmatizing

lemmatizer = Lemmatizer()
stopword = StopWord()

punctuation = list(string.punctuation)
stops = stopword.get_stopword() + punctuation + ['rt', 'via', '…','•','“', 'user', 'url',
                                                  'xf0', 'x9f', 'x98', 'x84', 'xf0', 'x9f', 'x98', 'x84', 'xf0', 'x9f', 
                                                  'x98', 'x84', 'xf', 'xe2', 'x80', 'x94', 'x82', 'xa4', 'x9d', 'xc2',
                                                  'x86', 'x85', 'x81', 'xa6', 'xa3', 'xad', 'x91', 'x81', 'xe3', 'x99',
                                                  'x', 'a', 'n', 'b',
                                                  '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

def remove_stops_and_lemmatize(tokenized_sentence):
  processed_words = []
  for word in tokenized_sentence:
    if word.lower() in kamus_alay.Alay.values:
      row = kamus_alay[kamus_alay['Alay'] == word.lower()]
      word = row['Tidak Alay'].values[0]
    if word.lower() not in stops:
      processed_words.append(lemmatizer.lemmatize(word))
  return processed_words

In [ ]:
def remove_stops_and_lemmatize_keep_exclamation(tokenized_sentence):
  processed_words = []
  for word in tokenized_sentence:
    if word == "!":
      processed_words.append(word)
    if word.lower() not in stops:
      processed_words.append(lemmatizer.lemmatize(word))
  return processed_words

## Prepare DF

In [ ]:
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
def set_placeholders(source_df):
  tweet = source_df["Tweet"]
  placeholder = [0 for x in range(len(tweet))]

  out_df = pd.DataFrame(tweet, columns=["Tweet"])
  out_df["Exclamation"] = placeholder
  out_df["Abusive"] = placeholder
  out_df["Length"] = placeholder
  out_df["Words"] = placeholder
  out_df["Uppercase"] = placeholder
  out_df["All Caps"] = placeholder

  for word in potensi_bias['0'].values:
      out_df[word] = placeholder
  for abusive in abusive_lexicon["ABUSIVE"].values:
      out_df[abusive] = placeholder
  for word in most_freq['0'].values:
      out_df[word] = placeholder

  out_df["HS"] = source_df["HS"]
  out_df["A"] = source_df["Abusive"]

  return out_df

### Train

In [ ]:
train_data = pd.concat([
    remaining_data,
    eda_data])

In [ ]:
train_df = set_placeholders(train_data)

### Test

In [ ]:
test_df = set_placeholders(test_data)

### SynTest

In [ ]:
syn_test_df = set_placeholders(synthetic_test_data)



```
# This is formatted as code
```

## Orthographic Features

### Train Data

In [ ]:
for i, row in train_df.iterrows():
  words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))
  ex_words = remove_stops_and_lemmatize_keep_exclamation(tokenize(row["Tweet"]))

  for xword in ex_words:
    if xword == "!":
      train_df.loc[i, 'Exclamation'] = 1
      break

  for word in words:
      if word.lower() in abusive_lexicon.values:
        train_df.loc[i, 'Abusive'] = 1

  length = len(row["Tweet"])
  train_df.loc[i, 'Length'] = length

  word_count = len(words)
  train_df.loc[i, 'Words'] = word_count

  uppercase = sum(1 for c in row["Tweet"] if c.isupper())
  train_df.loc[i, 'Uppercase'] = uppercase
  train_df.loc[i, 'All Caps'] = 1 if row["Tweet"].isupper() else 0
        

### Test + SynTest Data

In [ ]:
for i, row in test_df.iterrows():
  words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))
  ex_words = remove_stops_and_lemmatize_keep_exclamation(tokenize(row["Tweet"]))

  for xword in ex_words:
    if xword == "!":
      test_df.loc[i, 'Exclamation'] = 1
      break

  for word in words:
      if word.lower() in abusive_lexicon.values:
        test_df.loc[i, 'Abusive'] = 1

  length = len(row["Tweet"])
  test_df.loc[i, 'Length'] = length

  word_count = len(words)
  test_df.loc[i, 'Words'] = word_count

  uppercase = sum(1 for c in row["Tweet"] if c.isupper())
  test_df.loc[i, 'Uppercase'] = uppercase

  syn_test_df.loc[i*5, 'Uppercase'] = uppercase
  syn_test_df.loc[i*5 +1, 'Uppercase'] = uppercase
  syn_test_df.loc[i*5 +2, 'Uppercase'] = uppercase
  syn_test_df.loc[i*5 +3, 'Uppercase'] = uppercase
  syn_test_df.loc[i*5 +4, 'Uppercase'] = uppercase

  all_caps = 1 if row["Tweet"].isupper() else 0
  test_df.loc[i, 'All Caps'] = all_caps

  syn_test_df.loc[i*5, 'All Caps'] = all_caps
  syn_test_df.loc[i*5 +1, 'All Caps'] = all_caps
  syn_test_df.loc[i*5 +2, 'All Caps'] = all_caps
  syn_test_df.loc[i*5 +3, 'All Caps'] = all_caps
  syn_test_df.loc[i*5 +4, 'All Caps'] = all_caps
        

In [ ]:
for i, row in syn_test_df.iterrows():
  words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))
  ex_words = remove_stops_and_lemmatize_keep_exclamation(tokenize(row["Tweet"]))

  for xword in ex_words:
    if xword == "!":
      syn_test_df.loc[i, 'Exclamation'] = 1
      break

  for word in words:
    if word.lower() in abusive_lexicon.values:
      syn_test_df.loc[i, 'Abusive'] = 1
      break

  length = len(row["Tweet"])
  syn_test_df.loc[i, 'Length'] = length

  word_count = len(words)
  syn_test_df.loc[i, 'Words'] = word_count
        

## Word Features

Function

In [ ]:
def extract_by_word(df):
  for i, row in df.iterrows():
    words = remove_stops_and_lemmatize(tokenize(row["Tweet"]))

    for word in words:
      if word.lower() in potensi_bias["0"].values:
        df.loc[i, word.lower()] = 1
      if word.lower() in abusive_lexicon["ABUSIVE"].values:
        df.loc[i, word.lower()] = 1
      if word.lower() in most_freq["0"].values:
        df.loc[i, word.lower()] = 1
  
  return df

Extraction

In [ ]:
train_df = extract_by_word(train_df)
test_df = extract_by_word(test_df)
syn_test_df = extract_by_word(syn_test_df)

# Store df

In [ ]:
train_df["HS"] = train_data["HS"]
train_df["A"] = train_data["Abusive"]

test_df["HS"] = test_data["HS"]
test_df["A"] = test_data["Abusive"]

syn_test_df["HS"] = synthetic_test_data["HS"]
syn_test_df["A"] = synthetic_test_data["Abusive"]

In [ ]:
train_df

Tweet  Exclamation  Abusive  \
0     - disaat semua cowok berusaha melacak perhatia...            1        1   
1     Nah admin belanja satu lagi port terbaik nak m...            0        0   
2     Aktor huruhara 98 Prabowo S ingin lengserkan p...            0        1   
3     USER Bu guru enakan jadi jablay atau guru esde...            0        1   
4     USER USER USER USER USER USER Lawan bicara gw ...            0        1   
...                                                 ...          ...      ...   
3287  USER keluarga halilintar hmm masa km gatau. udik'            0        1   
3288  #2019GantiPresiden scra konstitusional hak set...            1        0   
3289  USER si ahok pantasnya dinobatkan sbgai Babi n...            0        1   
3290  YA ALLAH INI XL GAK KURANG KAMPRET LAGI APA KO...            0        1   
3291  USER Den...den...\nNista banget hidup lo..\nUd...            0        1   

      Length  Words  Uppercase  All Caps  jokowi  islam  indonesia  ...  \
0         74      6          2         0       0      0          0  ...   
1         77      6          4         0       0      1          0  ...   
2        135      9          6         0       1      0          0  ...   
3        133     11          4         0       0      0          0  ...   
4        174     13          2         0       0      0          0  ...   
...      ...    ...        ...       ...     ...    ...        ...  ...   
3287      49      6          4         0       0      0          0  ...   
3288     118     10          3         0       0      0          0  ...   
3289      52      6          5         0       0      0          0  ...   
3290      61      5         49         1       0      1          0  ...   
3291     232     22         12         0       0      0          0  ...   

      marah  iman  bangga  muka  paksa  lanjut  bandara  menteri  HS  A  
0         0     0       0     0      0       0        0        0   1  1  
1         0     0       0     0      0       0        0        0   0  0  
2         0     0       0     0      0       0        0        0   1  0  
3         0     0       0     0      0       0        0        0   1  1  
4         0     0       0     0      0       0        0        0   1  1  
...     ...   ...     ...   ...    ...     ...      ...      ...  .. ..  
3287      0     0       0     0      0       0        0        0   1  1  
3288      0     0       0     0      0       0        0        0   1  1  
3289      0     0       0     0      0       0        0        0   1  1  
3290      0     0       0     0      0       0        0        0   1  1  
3291      0     0       0     0      0       0        0        0   1  1  

[9219 rows x 392 columns]

In [ ]:
test_df

Tweet  Exclamation  Abusive  \
0     USER Ooo camtu.. Biasalah kena seimbang.. Ada ...            0        1   
1     Presiden Minta Sistem Perizinan Berusaha Terin...            0        0   
2     ð???ð???ð???tak berambisi? Mimin bisa aja judu...            0        0   
3     USER Kemaren gubernur pilihan umat impor bawan...            0        1   
4     4. Apakah pernah mengalami kendala dalam menga...            0        0   
...                                                 ...          ...      ...   
3945  Doi tuh native speaker ya.\n\nMother language-...            0        1   
3946  RT USER Kasian ya kacung tetap kacung anies sandi            0        0   
3947  USER kacrut , rank 10 tapi mainnya gk cerdas a...            0        1   
3948  USER Klau kmu di banding AR jauhlah.koen menun...            0        1   
3949  Mana para babi ahok yg ngejek onta, biar di in...            0        1   

      Length  Words  Uppercase  All Caps  jokowi  islam  indonesia  ...  \
0        276     29         12         0       0      0          0  ...   
1        164     19         19         0       0      0          0  ...   
2         53      8          1         0       0      0          0  ...   
3         93     11          5         0       0      0          0  ...   
4         69      5          7         0       0      0          0  ...   
...      ...    ...        ...       ...     ...    ...        ...  ...   
3945      88      8          3         0       0      0          0  ...   
3946      49      6          7         0       0      0          0  ...   
3947      62      7          4         0       0      0          0  ...   
3948     219     23          7         0       0      0          0  ...   
3949      59      7          1         0       0      0          0  ...   

      marah  iman  bangga  muka  paksa  lanjut  bandara  menteri  HS  A  
0         0     0       0     0      0       0        0        0   0  0  
1         0     0       0     0      0       0        0        0   0  0  
2         0     0       0     0      0       0        0        0   0  0  
3         0     0       0     0      0       0        0        0   0  0  
4         0     0       0     0      0       0        0        0   0  0  
...     ...   ...     ...   ...    ...     ...      ...      ...  .. ..  
3945      0     0       0     0      0       0        0        0   1  1  
3946      0     0       0     0      0       0        0        0   1  1  
3947      0     0       0     0      0       0        0        0   1  1  
3948      0     0       0     0      0       0        0        0   1  1  
3949      0     0       0     0      0       0        0        0   1  1  

[3950 rows x 392 columns]

In [ ]:
syn_test_df

Tweet  Exclamation  \
0      user ooo camtu.. biasalah kena seimbang.. ada ...            0   
1      user ooo camtu.. biasalah kena seimbang.. ada ...            0   
2      user ooo camtu.. biasalah kena seimbang.. ada ...            0   
3      user ooo camtu.. biasalah kena seimbang.. ada ...            0   
4      user ooo camtu.. biasalah kena seimbang.. ada ...            0   
...                                                  ...          ...   
19745  mana para babi adi yg ngejek onta, biar di inj...            0   
19746  mana para babi doni yg ngejek onta, biar di in...            0   
19747  mana para babi zaidan yg ngejek onta, biar di ...            0   
19748  mana para babi baraz yg ngejek onta, biar di i...            0   
19749  mana para babi usman yg ngejek onta, biar di i...            0   

       Abusive  Length  Words  Uppercase  All Caps  jokowi  islam  indonesia  \
0            1     276     29         12         0       0      0          0   
1            1     276     29         12         0       0      0          0   
2            1     276     29         12         0       0      0          0   
3            1     276     29         12         0       0      0          0   
4            1     276     29         12         0       0      0          0   
...        ...     ...    ...        ...       ...     ...    ...        ...   
19745        1      58      7          1         0       0      0          0   
19746        1      59      7          1         0       0      0          0   
19747        1      61      7          1         0       0      0          0   
19748        1      60      7          1         0       0      0          0   
19749        1      60      7          1         0       0      0          0   

       ...  marah  iman  bangga  muka  paksa  lanjut  bandara  menteri  HS  A  
0      ...      0     0       0     0      0       0        0        0   0  0  
1      ...      0     0       0     0      0       0        0        0   0  0  
2      ...      0     0       0     0      0       0        0        0   0  0  
3      ...      0     0       0     0      0       0        0        0   0  0  
4      ...      0     0       0     0      0       0        0        0   0  0  
...    ...    ...   ...     ...   ...    ...     ...      ...      ...  .. ..  
19745  ...      0     0       0     0      0       0        0        0   1  1  
19746  ...      0     0       0     0      0       0        0        0   1  1  
19747  ...      0     0       0     0      0       0        0        0   1  1  
19748  ...      0     0       0     0      0       0        0        0   1  1  
19749  ...      0     0       0     0      0       0        0        0   1  1  

[19750 rows x 392 columns]

In [ ]:
from google.colab import files

train_df.to_csv('all_extracted_train_data.csv', encoding = 'latin-1') 
files.download('all_extracted_train_data.csv')

test_df.to_csv('all_extracted_test_data.csv', encoding = 'latin-1') 
files.download('all_extracted_test_data.csv')

syn_test_df.to_csv('all_extracted_synthetic_test_data.csv', encoding = 'latin-1') 
files.download('all_extracted_synthetic_test_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>